# Tools and Routing with LangChain

In [4]:
from langchain.agents import tool
from pydantic import BaseModel, Field

class SearchQuery(BaseModel):
    query: str = Field(description="The query to search for")

@tool(args_schema=SearchQuery)
def search(query: str) -> str:
    """Search the web for information about the query."""
    return f"Search results for {query}"

print("name:", search.name, ", description:", search.description)
search.args

name: search , description: Search the web for information about the query.


{'query': {'description': 'The query to search for',
  'title': 'Query',
  'type': 'string'}}

In [5]:
search.run('anything')

'Search results for anything'

## Tool to get the current temperature by lat and long

In [6]:
import requests
from typing import Optional

class TemperatureQuery(BaseModel):
    latitude: float = Field(description="The latitude coordinate")
    longitude: float = Field(description="The longitude coordinate")

@tool(args_schema=TemperatureQuery)
def get_current_temperature(latitude: float, longitude: float) -> str:
    """Get the current temperature for a given latitude and longitude using Open-Meteo API."""
    try:
        # Open-Meteo API endpoint for current weather
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "current": "temperature_2m",
            "temperature_unit": "celsius"
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        data = response.json()
        current_temp = data["current"]["temperature_2m"]
        
        return f"Current temperature is {current_temp}°C"
    
    except Exception as e:
        return f"Error getting temperature: {str(e)}"

# Test the function
print("name:", get_current_temperature.name, ", description:", get_current_temperature.description)
get_current_temperature.args


name: get_current_temperature , description: Get the current temperature for a given latitude and longitude using Open-Meteo API.


{'latitude': {'description': 'The latitude coordinate',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'The longitude coordinate',
  'title': 'Longitude',
  'type': 'number'}}

In [7]:
get_current_temperature.run({"latitude": 37.7749, "longitude": -122.4194})

'Current temperature is 17.1°C'

## Tool to get summary of wikipedia page matching the query passed

In [19]:
import wikipedia
from typing import List, Dict

class WikipediaQuery(BaseModel):
    query: str = Field(description="The search query for Wikipedia")

@tool(args_schema=WikipediaQuery)
def search_wikipedia(query: str) -> str:
    """Search Wikipedia and return top 3 page titles and summaries for a given query."""
    try:
        # Search for pages matching the query
        search_results = wikipedia.search(query, results=3)
        
        if not search_results:
            return f"No Wikipedia pages found for query: {query}"
        
        results = []
        for title in search_results:
            try:
                # Get summary for each page
                summary = wikipedia.summary(title, sentences=2)
                results.append(f"**{title}**: {summary}")
            except wikipedia.exceptions.DisambiguationError as e:
                # If disambiguation page, use the first option
                try:
                    summary = wikipedia.summary(e.options[0], sentences=2)
                    print(f"**{e.options[0]}**: {summary}")
                except:
                    print(f"**{title}**: Could not retrieve summary")
            except wikipedia.exceptions.PageError:
                print(f"**{title}**: Page not found")
            except Exception as e:
                print(f"**{title}**: Error retrieving summary")
        
        return "\n\n".join(results)
    
    except Exception as e:
        return f"Error searching Wikipedia: {str(e)}"

# Test the function
print("name:", search_wikipedia.name, ", description:", search_wikipedia.description)
search_wikipedia.args

name: search_wikipedia , description: Search Wikipedia and return top 3 page titles and summaries for a given query.


{'query': {'description': 'The search query for Wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [9]:
search_wikipedia.run("France")

**France**: Page not found


/Users/n0c09jf/miniconda3/envs/langchain/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/n0c09jf/miniconda3/envs/langchain/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


**French**: Could not retrieve summary


'**French of France**: French of France (French: français de France [fʁɑ̃sɛ də fʁɑ̃s]) is the predominant variety of the French language in France, Andorra and Monaco, in its formal and informal registers. It has, for a long time, been associated with Standard French.'

## Tools based on Open API 3.0 specification

In [76]:
# TEMPORARILY COMMENTED OUT DUE TO PYDANTIC COMPATIBILITY ISSUE
# The OpenAPISpec functionality has a compatibility issue with current Pydantic versions
# This is a known issue: 'super' object has no attribute 'parse_obj'
# 
# from langchain_community.utilities.openapi import OpenAPISpec
# # from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
# #from langchain_community.agent_toolkits.openapi import planner
# 
# spec = OpenAPISpec.from_url("https://petstore3.swagger.io/api/v3/openapi.json")
# spec.get_path_parameters("get", "/pets/{pet_id}")

print("✅ OpenAPISpec section temporarily disabled due to Pydantic compatibility issue.")
print("📝 The rest of the notebook should work fine with the custom tools we created above.")
print("🔧 To fix this issue, you would need to:")
print("   1. Use Pydantic v1 with older LangChain versions, or")
print("   2. Wait for openapi-schema-pydantic to be updated for Pydantic v2 compatibility")

✅ OpenAPISpec section temporarily disabled due to Pydantic compatibility issue.
📝 The rest of the notebook should work fine with the custom tools we created above.
🔧 To fix this issue, you would need to:
   1. Use Pydantic v1 with older LangChain versions, or
   2. Wait for openapi-schema-pydantic to be updated for Pydantic v2 compatibility


## Provide tools to LLM and converse

In [61]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.agents.output_parsers.tools import ToolAgentAction
from langchain_core.agents import AgentFinish
from langchain_core.messages import AIMessage


llm = ChatOpenAI(
    temperature=0, 
    base_url="http://localhost:1234/v1", 
    api_key="dummy-key",
    model="qwen/qwen3-4b-2507") # [deepseek/deepseek-r1-0528-qwen3-8b, meta-llama-3.1-8b-instruct]

llm_with_tools = llm.bind_tools([search_wikipedia, get_current_temperature])

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can search Wikipedia and get the current temperature of a city. DONOT use tools for grettings."),
    ("human", "{input}")
])

def print_result(result):
    print(result)
    print(type(result))
    if isinstance(result, list) and len(result) > 0:
        result = result[0]

    if isinstance(result, AgentFinish):
        print("AgentFinish Output:", result.return_values["output"])

    elif isinstance(result, ToolAgentAction):
        print("ToolAgentAction Tool:", result.tool)
        print("ToolAgentAction Tool Input:", result.tool_input)
    elif isinstance(result, AIMessage):
        print("AIMessageContent:", result.content)
        print("AIMessageToolCalls:", result.tool_calls)

chain = prompt | llm_with_tools

response = chain.invoke({"input": "get articles on Bitcoin"})
print_result(response)

content='' additional_kwargs={'tool_calls': [{'id': '588280792', 'function': {'arguments': '{"query":"Bitcoin"}', 'name': 'search_wikipedia'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 294, 'total_tokens': 315, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen/qwen3-4b-2507', 'system_fingerprint': 'qwen/qwen3-4b-2507', 'id': 'chatcmpl-y28df0c08cmnisdw6f59k', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run--60c9143a-34c0-42a9-8600-cd43b9bc7fa7-0' tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'Bitcoin'}, 'id': '588280792', 'type': 'tool_call'}] usage_metadata={'input_tokens': 294, 'output_tokens': 21, 'total_tokens': 315, 'input_token_details': {}, 'output_token_details': {}}
<class 'langchain_core.messages.ai.AIMessage'>
AIMessageContent: 
AIMessageToolCalls: [{'name': 'search_wikipedia', 'args': {'query': 'Bitcoin'}, 'id': '588280792

In [62]:
response = chain.invoke({"input": "What is the current temperature in SF?"})
print_result(response)

content='' additional_kwargs={'tool_calls': [{'id': '295552382', 'function': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 298, 'total_tokens': 338, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen/qwen3-4b-2507', 'system_fingerprint': 'qwen/qwen3-4b-2507', 'id': 'chatcmpl-zzmdm1xlwxlhyuptd0rh', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run--434b2f64-0937-40b4-8033-145d92c0302b-0' tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': '295552382', 'type': 'tool_call'}] usage_metadata={'input_tokens': 298, 'output_tokens': 40, 'total_tokens': 338, 'input_token_details': {}, 'output_token_details': {}}
<class 'langchain_core.messages.ai.AIMessage'>
AIMessageContent: 
AIMessageToolCalls: [{'name': 'get_

In [63]:
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
# from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

openai_chain = chain | OpenAIToolsAgentOutputParser()

result = openai_chain.invoke({"input": "What is the capital of France?"})

print_result(result)

return_values={'output': 'The capital of France is Paris.'} log='The capital of France is Paris.'
<class 'langchain_core.agents.AgentFinish'>
AgentFinish Output: The capital of France is Paris.


In [64]:
result = openai_chain.invoke({"input": "What is the current temperature in SF?"})
print_result(result)

[ToolAgentAction(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '209038887', 'function': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 298, 'total_tokens': 338, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen/qwen3-4b-2507', 'system_fingerprint': 'qwen/qwen3-4b-2507', 'id': 'chatcmpl-hvsibwgjingzgyklrsfod', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2040b235-3838-4eb8-9e6c-f84bf2558398-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': '209038887', 'type': 'tool_call

In [67]:
get_current_temperature(result[0].tool_input)

'Current temperature is 17.0°C'

In [68]:
result = openai_chain.invoke({"input": "Hi!"}) 
print_result(result)

return_values={'output': 'Hello! How can I assist you today?'} log='Hello! How can I assist you today?'
<class 'langchain_core.agents.AgentFinish'>
AgentFinish Output: Hello! How can I assist you today?


## Routing the call to tool

In [70]:
from langchain_core.runnables import RunnableLambda

def route(x):
    if isinstance(x, list):
        x = x[0]

    if isinstance(x, AgentFinish):
        return x.return_values["output"]
    elif isinstance(x, ToolAgentAction):
        tool_name = x.tool 
        if tool_name == "search_wikipedia":
            return search_wikipedia(x.tool_input)
        elif tool_name == "get_current_temperature":
            return get_current_temperature(x.tool_input)

    return ""

router = RunnableLambda(lambda x: route(x))
complete_chain = openai_chain | router

In [71]:
complete_chain.invoke({"input": "What is the current temperature in SF?"})

'Current temperature is 17.4°C'

In [72]:
complete_chain.invoke({"input": "get articles on Bitcoin"})

"**Bitcoin**: Bitcoin (abbreviation: BTC; sign: ₿) is the first decentralized cryptocurrency. Based on a free-market ideology, bitcoin was invented in 2008 when an unknown entity published a white paper under the pseudonym of Satoshi Nakamoto.\n\n**History of bitcoin**: Bitcoin is a cryptocurrency, a digital asset that uses cryptography to control its creation and management rather than relying on central authorities. Originally designed as a medium of exchange, Bitcoin is now primarily regarded as a store of value.\n\n**Strategic bitcoin reserve (United States)**: The strategic bitcoin reserve is a reserve asset, funded by the United States Treasury's forfeited bitcoin, announced by President Donald Trump in March 2025. Separately, a digital asset stockpile for non-bitcoin assets was also created."

In [73]:
complete_chain.invoke({"input": "Hi!"})

'Hello! How can I assist you today?'

In [74]:
complete_chain.invoke({"input": "What is the captial of France?"})

'The capital of France is Paris.'